In [1]:
import numpy as np, pandas as pd, random
from tqdm import tqdm

# --------------------------------------------------------
# 1) carica
# --------------------------------------------------------
df = pd.read_parquet("positions.parquet")

# --------------------------------------------------------
# 3) pesi
# --------------------------------------------------------
w_path, w_wall, w_goal, w_deg = 0.2, 0.1, 0.2, 0.5

# --------------------------------------------------------
# 4) score (minore è migliore)
# --------------------------------------------------------
df["total_score"] = (
      w_path * df.path_distance        #  + penalità: cammino lungo
    + w_goal * df.distance_to_goal        #  + penalità: lontano dal goal
    - w_deg  * df.degree         #  + penalità: vicolo cieco
    - w_wall * df.distance_from_wall        #  – bonus: punti “larghi”, lontani dai muri
)

# quick sanity
print(df.nsmallest(3,"total_score")[["x","y","degree","total_score"]])
print(df.nlargest (3,"total_score")[["x","y","degree","total_score"]])

# --------------------------------------------------------
# 5) genera 200k coppie (50 % facili / 50 % difficili)
# --------------------------------------------------------
from tqdm import tqdm
import numpy as np
import pandas as pd

def sample_pairs(
        df: pd.DataFrame,
        num_pairs: int        = 200_000,
        hard_ratio: float     = 0.7,
        thresh: float         = 0.05,
        min_delta_deg: float  = 0.001,     # <- NOVITÀ: differenza minima di degree
        max_trials: int       = 50_000_000   # per evitare loop infiniti
    ):
    """
    Estrae `num_pairs` coppie (x_better,y_better,x_worse,y_worse)
    rispettando:
      • hard_ratio      → % di coppie 'difficili' (score simili)
      • thresh          → soglia fra facile / difficile su total_score
      • min_delta_deg   → scarto minimo assoluto su colonna 'degree'
    Restituisce una lista di tuple di lunghezza 5 (… , 1) compatibili
    con il tuo formato.
    """

    idx       = df.index.to_numpy()
    scores    = df["total_score"].to_numpy()
    degrees   = df["degree"].to_numpy()          # cache per velocità
    pairs     = set()
    n_hard    = int(num_pairs * hard_ratio)

    pbar = tqdm(total=num_pairs, desc="sampling pairs")
    trials = 0
    while len(pairs) < num_pairs and trials < max_trials:
        trials += 1

        i, j = np.random.choice(idx, 2, replace=False)
        si, sj = scores[i], scores[j]

        if si == sj:          # identico score ⇒ salta subito
            continue

        # -----------------------------------------  
        # 1) vincolo sullo score facile / difficile
        # -----------------------------------------
        better, worse = (i, j) if si < sj else (j, i)   # score minore = migliore
        delta_score   = abs(si - sj)
        want_hard     = len(pairs) < n_hard
        if  (want_hard  and delta_score >= thresh) or \
            (not want_hard and delta_score <  thresh):
            continue        # scorretto per la fascia che stiamo riempiendo

        # -----------------------------------------  
        # 2) vincolo sullo scarto di degree
        # -----------------------------------------
        if abs(degrees[better] - degrees[worse]) < min_delta_deg:
            continue        # differenza troppo piccola → poco informativa

        # -----------------------------------------  
        # 3) aggiungi la coppia
        # -----------------------------------------
        pair = (df.at[better,"x"], df.at[better,"y"],
                df.at[worse, "x"], df.at[worse, "y"], 1)

        if pair not in pairs:
            pairs.add(pair)
            pbar.update(1)

    pbar.close()

    if len(pairs) < num_pairs:
        print(f"⚠️  solo {len(pairs)} coppie generate dopo {trials} tentativi "
              f"(probabilmente min_delta_deg è troppo alto).")

    return list(pairs)


pairs = sample_pairs(df, num_pairs=200_000, hard_ratio=0.7)

# --------------------------------------------------------
# 6) salva
# --------------------------------------------------------
pd.DataFrame(pairs,
    columns=["x_better","y_better","x_worse","y_worse","preference"]
).to_parquet("preferences.parquet", index=False)

print("✓ preferences.parquet salvato con", len(pairs), "coppie")


             x         y    degree  total_score
564   0.932393  0.949949  0.912974    -0.534728
1983  0.939920  0.931047  0.893655    -0.525333
1798  0.930260  0.939053  0.888121    -0.520514
             x         y    degree  total_score
2365  0.007552  0.102551  0.000000     0.379665
2091  0.004755  0.043673  0.028194     0.373854
2874  0.058836  0.094060  0.002973     0.373508


sampling pairs: 100%|██████████| 200000/200000 [00:46<00:00, 4298.02it/s] 


✓ preferences.parquet salvato con 200000 coppie
